In [448]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchtext.legacy.data import Field,BucketIterator,TabularDataset
from torchtext.legacy import data
import torchtext
import spacy
import re
from nltk.tokenize import word_tokenize
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from tqdm import tqdm
from torchtext.vocab import Vectors, GloVe, FastText

nltk.download('punkt')
%matplotlib inline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pal = sns.color_palette()

[nltk_data] Downloading package punkt to /Users/zhangyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv("test.csv")
print("the length of train set",len(data_train))
print("the length of test set", len(data_test))

the length of train set 404290
the length of test set 2345796


In [380]:
train = data_train[:50000]
validation = data_train[50000:52000]
test = data_test
# [i for i in data_train.question1]

In [381]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am",'i\'m':'i am', "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled'}

In [382]:
def clean_contractions(text, mapping):
    text = text.lower()
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else mapping[t.lower()] if t.lower() in mapping else t for t in text.split(" ")])
    return text

In [383]:
def remove_newlines(sent):
    sent = re.sub(r'\s+', " ", sent )
    return sent

In [384]:
train['question1'] = train['question1'].apply(lambda x: clean_contractions(str(x),contraction_mapping))
train['question2'] = train['question2'].apply(lambda x: clean_contractions(str(x),contraction_mapping))
validation['question1'] = validation['question1'].apply(lambda x: clean_contractions(str(x),contraction_mapping))
validation['question2'] = validation['question2'].apply(lambda x: clean_contractions(str(x),contraction_mapping))

train['question1'] = train['question1'].apply(lambda x: remove_newlines(str(x)))
train['question2'] = train['question2'].apply(lambda x: remove_newlines(str(x)))
validation['question1'] = validation['question1'].apply(lambda x: remove_newlines(str(x)))
validation['question2'] = validation['question2'].apply(lambda x: remove_newlines(str(x)))
test['question1'] = test['question1'].apply(lambda x: clean_contractions(str(x),contraction_mapping))
test['question2'] = test['question2'].apply(lambda x: clean_contractions(str(x),contraction_mapping))

test['question1'] = test['question1'].apply(lambda x: remove_newlines(str(x)))
test['question2'] = test['question2'].apply(lambda x: remove_newlines(str(x)))


/Users/zhangyan/anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/zhangyan/anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/zhangyan/anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [385]:
labels = train['is_duplicate']

In [386]:
validation.drop(['id','qid1','qid2'],inplace=True,axis = 1)
train.drop(['id','qid1','qid2'],inplace=True,axis = 1)
test.drop(['test_id'],inplace=True,axis = 1)

/Users/zhangyan/anaconda/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


KeyError: "['test_id'] not found in axis"

In [387]:
train.to_csv('data/Train.csv',index = False)
validation.to_csv('data/Validation.csv',index = False)
test.to_csv('data/Test.csv',index = False)

In [388]:
TEXT1 = data.Field(tokenize='spacy', batch_first=True, lower=True, include_lengths=True)
TEXT2 = data.Field(tokenize='spacy', batch_first=True, lower=True, include_lengths=True)
LABEL = data.LabelField(dtype=torch.float)

In [389]:
fields1 = [('question1',TEXT1),('question2',TEXT2),('is_duplicate',LABEL)]
fields2 = [('question1',TEXT1),('question2',TEXT2)]

In [390]:
train_data, val_data = data.TabularDataset.splits(
    path='data/',
    train='Train.csv',
    validation = 'Validation.csv',
    format='csv',
    fields=fields1,
    skip_header=True
)

test_data = data.TabularDataset(
        path='data/Test.csv', format='csv',
        skip_header=True,
        fields=fields2
    )

In [451]:
TEXT1.build_vocab(
        train_data, val_data, test_data, max_size = 25000  # We use it for getting vocabulary of words
    )
TEXT2.build_vocab(
        train_data, val_data, test_data, max_size = 25000   # We use it for getting vocabulary of words
    )
LABEL.build_vocab(train_data,val_data)

In [452]:
# word_embeddings = TEXT1.vocab.vectors
# vocab_size = len(TEXT1.vocab)

def get_iterator(dataset, batch_size, train=True,
                 shuffle=True, repeat=False):
    
    device = torch.device('cuda:0' if torch.cuda.is_available()
                          else 'cpu')
    
    dataset_iter = data.Iterator(
        dataset, batch_size=batch_size, device=device,
        train=train, shuffle=shuffle, repeat=repeat,
        sort=False
    )
    
    return dataset_iter
batchsize = 64

train_iter = get_iterator(train_data, batch_size=batchsize, 
                              train=True, shuffle=True,
                              repeat=False)
val_iter = get_iterator(val_data, batch_size=batchsize, 
                        train=True, shuffle=True,
                        repeat=False)
test_iter = get_iterator(test_data, batch_size=batchsize, 
                             train=False, shuffle=False,
                             repeat=False)

In [453]:
print(len(TEXT1.vocab))
print(len(TEXT2.vocab))

25002
25002


In [454]:
print(TEXT1.vocab.freqs.most_common(20))

[('?', 2520795), ('the', 1027345), ('what', 943820), ('is', 832413), ('i', 644402), ('how', 633806), ('a', 608286), ('in', 596840), ('to', 569904), ('do', 451711), ('of', 445111), ('are', 398318), ('and', 372856), ('can', 335235), ('for', 314164), (',', 293943), ('why', 235395), ('you', 231209), ('it', 203235), ('my', 198991)]


In [461]:
class Model(nn.Module):
    def __init__(self, batch_size, bidirectional, num_layers, vocab_size, embedding_dim, hidden_dim,linear_dim2,linear_dim3, output_dim):
        super().__init__()
        self.batch_size = batch_size
        self.bidirectional = bidirectional
        self.hidden_size = hidden_dim
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first = True, dropout=0.3, bidirectional = self.bidirectional, num_layers = self.num_layers)
        self.num_directions  = (1 if self.bidirectional == False else 2)
        self.concat_layer = nn.Linear(2*self.num_directions*self.hidden_size, linear_dim2) #for concatenating both the outputs
        self.bn3 = nn.BatchNorm1d(linear_dim2,eps=1e-05, momentum=0.1, affine=True)
        self.fc1 = nn.Linear(linear_dim2, linear_dim3)#for prediction
        self.fc2 = nn.Linear(linear_dim3, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, text1, text2, question1_length, question2_length):
        embedded1 = self.embedding(text1)
#         embedded1 = nn.utils.rnn.pack_padded_sequence(embedded1.cpu(), question1_length)
        embedded2 = self.embedding(text2)
#         embedded2 = nn.utils.rnn.pack_padded_sequence(embedded2.cpu(), question2_length)
        output_1,(h_n_1,c_n_1) = self.lstm(embedded1)
        output_2,(h_n_2,c_n_2) = self.lstm(embedded2)
        
        output_1 = output_1[:,-1,:].view(self.batch_size,1, self.num_directions*self.hidden_size)
        output_2 = output_2[:,-1,:].view(self.batch_size,1,self.num_directions*self.hidden_size)
        concatenated_outputs = torch.cat((output_1,output_2),dim = 2)
        concatenated_outputs = concatenated_outputs.view(concatenated_outputs.shape[0],concatenated_outputs.shape[2]) 
        if model.train():
            concatenated_outputs = nn.Dropout(p=0.3)(concatenated_outputs)
#         concatenated_outputs = self.relu(concatenated_outputs)
        concatenated_outputs = self.concat_layer(concatenated_outputs)
#         concatenated_outputs = self.relu(concatenated_outputs)
        if model.train():
            concatenated_outputs = nn.Dropout(p=0.3)(concatenated_outputs)
        concatenated_outputs = self.fc1(concatenated_outputs)
#         concatenated_outputs = self.relu(concatenated_outputs)
        if model.train():
            concatenated_outputs = nn.Dropout(p=0.3)(concatenated_outputs)
        predictions = self.fc2(concatenated_outputs)
        return predictions
        
        

In [462]:
model = Model(batch_size = batchsize,
             bidirectional = True,
             num_layers = 1,
             vocab_size = len(TEXT1.vocab),
             embedding_dim = 100,
             hidden_dim = 50,
             linear_dim2 = 32,
             linear_dim3 = 16,
             output_dim = 1).to(device)

In [463]:
criterion = nn.BCEWithLogitsLoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate) 
epochs = 10

In [464]:
def return_accuracy(logits,label):
    sigmoid = nn.Sigmoid()(logits)
    predictions = torch.round(sigmoid)
    predictions = predictions.view(batchsize)
    return (predictions == label).sum().float()/float(label.size(0))

def train(epochs,criterion,optimizer,model,train_iterator,valid_iterator):
    
    for epoch in range(epochs):
        print("Epoch {} out of {}".format(epoch,epochs))
        
        epoch_train_loss = 0
        epoch_train_accuracy = 0
        
        epoch_valid_loss = 0
        epoch_valid_accuracy = 0
        model.train()
        for batch in tqdm(train_iterator):
            model.train()
            optimizer.zero_grad()
            question1 = batch.question1[0]
            question2 = batch.question2[0]
            label = batch.is_duplicate
            
            question1_length = batch.question1[1]
            question2_length = batch.question2[1]
            if len(batch.question1[1]) != batchsize:continue
            question1.to(device)
            question2.to(device)
            label.to(device)

            label = torch.tensor(label,dtype= torch.float32,device = device)
            
            predictions = model(question1,question2,question1_length,question2_length)
            loss = criterion(predictions,label.unsqueeze(1))
            
            loss.backward()
            optimizer.step()
            
            batch_loss = loss.item()/len(batch)
            batch_accuracy = return_accuracy(predictions,label)
            
            epoch_train_loss += loss.item()
            epoch_train_accuracy += batch_accuracy.item()
            
        print("Epoch Train Accuracy: ",epoch_train_accuracy/len(train_iterator))
        print("Epoch Train Loss: ",epoch_train_loss/len(train_iterator))
        model.eval()
        for batch in tqdm(valid_iterator):
            model.eval()
            question1_v = batch.question1[0]
            question2_v = batch.question2[0]
            label_v = batch.is_duplicate
            
            question1_length_v = batch.question1[1]
            question2_length_v = batch.question2[1]
            if len(question1_length_v) != batchsize:continue
            question1_v.to(device)
            question2_v.to(device)
            label_v.to(device)

            label_v = torch.tensor(label_v,dtype= torch.float32,device = device)
            
            predictions_v = model(question1_v,question2_v,question1_length_v,question2_length_v)
            loss_v = criterion(predictions_v,label_v.unsqueeze(1))
            
            batch_loss_v = loss_v.item()/len(batch)
            batch_accuracy_v = return_accuracy(predictions_v,label_v)
            
            epoch_valid_loss += loss_v.item()
            epoch_valid_accuracy += batch_accuracy_v.item()
            
        print("Epoch valid Accuracy: ",epoch_valid_accuracy/len(valid_iterator))
        print("Epoch valid Loss: ",epoch_valid_loss/len(valid_iterator))
        print("--"*60) 

In [465]:
train(epochs,criterion,optimizer,model,train_iter,val_iter)

  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 0 out of 10


/Users/zhangyan/anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  9%|▉         | 3/32 [00:00<00:01, 22.82it/s]

Epoch Train Accuracy:  0.6650415601023018
Epoch Train Loss:  0.6161656396849381


  0%|          | 1/782 [00:00<02:33,  5.08it/s]

...

Epoch Train Accuracy:  0.8068853900255755
Epoch Train Loss:  0.43158045102415793


  0%|          | 1/782 [00:00<02:03,  6.30it/s]

Epoch valid Accuracy:  0.68603515625
Epoch valid Loss:  0.5732296807691455
------------------------------------------------------------------------------------------------------------------------
Epoch 8 out of 10


 28%|██▊       | 216/782 [00:26<01:09,  8.09it/s]


KeyboardInterrupt: 

In [ ]:

def predict(test_iterator,model):
    preds = torch.tensor([])
    epoch_test_loss = 0
    epoch_test_accuracy = 0
    for batch in tqdm(test_iterator):
        model.eval()
        question1 = batch.question1[0]
        question2 = batch.question2[0]
        question1_length = batch.question1[1]
        question2_length = batch.question2[1]

        question1.to(device)
        question2.to(device)
        predictions = model(question1,question2,question1_length,question2_length)
        sigmoid = nn.Sigmoid()(predictions)
        predictions = torch.round(sigmoid)
        predictions = predictions.view(batchsize)
        preds = torch.cat((preds, predictions), axis=0)

    return preds


In [ ]:

preds = predict(test_iter, model)
print(preds)